In [44]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [45]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [51]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 100
n_iter = 13000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [52]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
import impl.NN as nn

class RNN(nn.NN):

    def __init__(self, D, H, char2idx, idx2char):
        self.D = D
        self.H = H
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.vocab_size = len(char2idx)
        super().__init__(D, D, H, None, None, loss='cross_ent', nonlin='relu')

    def initial_state(self):
        return np.zeros((1, self.H))

    def forward(self, X, h, m): # m = self.model
        Wxh, Whh, Why = m['Wxh'], m['Whh'], m['Why']
        bh, by = m['bh'], m['by']

        hprev = h.copy()

        h, h_cache = l.tanh_forward(X @ Wxh + hprev @ Whh + bh)
        y, y_cache = l.fc_forward(h, Why, by)

        cache = X, Whh, h, hprev, y, h_cache, y_cache, Wxh

        return y, h, cache

    def backward(self, dy, dh, cache):
        X, Whh, h, hprev, y, h_cache, y_cache, Wxh = cache

        dh_next = dh.copy()
        
        # Hidden to output gradient
        dh, dWhy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dby = dby.reshape((1, -1))

        # tanh
        dh = l.tanh_backward(dh, h_cache)

        # Hidden gradient
        dbh = dh
        dWhh = hprev.T @ dh
        dWxh = X.T @ dh
        
        dX = dh @ Wxh.T
        dh = dh @ Whh.T

        grad = dict(Wxh=dWxh, Whh=dWhh, Why=dWhy, bh=dbh, by=dby)
        
        return dX, dh, grad

    def _init_model(self, D, C, H):
        self.model = []
        for l in range(4):
            m = dict(
                Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
                Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
                Why=np.random.randn(H, D) / np.sqrt(C / 2.),
                bh=np.zeros((1, H)),
                by=np.zeros((1, D))
                )
            self.model.append(m)

    def train_step_fwd(self, X_train, h):
        ys , caches = [], []
        h_init = h.copy()
        h = []
        for l in range(4):
            h.append(h_init.copy())
            caches.append([])
            
        for X in X_train:
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            y = X_one_hot.reshape(1, -1)
            for l in range(4):
                y, h[l], cache = self.forward(y, h[l], self.model[l])
                caches[l].append(cache)
                
            ys.append(y)
            
        return ys, caches

    def train_step_bwd(self, y_train, ys, caches):
        loss, dys = 0.0, []

        for y_pred, y in zip(ys, y_train):
            loss += loss_fun.cross_entropy(self.model[0], y_pred, y, lam=0)/ y_train.shape[0]
            dy = loss_fun.dcross_entropy(y_pred, y)
            dys.append(dy)

        # Grads
        dh, grad, grads = [], [], []
        for l in range(4):
            dh.append(np.zeros((1, self.H)))
            grad.append({key: np.zeros_like(val) for key, val in self.model[0].items()})
            grads.append({key: np.zeros_like(val) for key, val in self.model[0].items()})
            
        for t in reversed(range(len(dys))):
            dX = dys[t]
            for l in reversed(range(4)):
                dX, dh[l], grad[l] = self.backward(dX, dh[l], caches[l][t])
                for k in grad[0].keys():
                    grads[l][k] += grad[l][k]
                
#             dX, dh[3], grad[3] = self.backward(dys[t], dh[3], caches[3][t]) # 4th layer 
#             dX, dh[2], grad[2] = self.backward(dX, dh[2], caches[2][t]) # 3rd layer 
#             dX, dh[1], grad[1] = self.backward(dX, dh[1], caches[1][t]) # 2nd layer 
#             dX, dh[0], grad[0] = self.backward(dX, dh[0], caches[0][t]) # first layer
#             for k in grad[0].keys():
#                 grads[0][k] += grad[0][k]
#                 grads[1][k] += grad[1][k]
#                 grads[2][k] += grad[2][k]
#                 grads[3][k] += grad[3][k]
                
        return loss, grads
    
    def test(self, X_seed, h, size=100):
        chars = [self.idx2char[X_seed]]
        idx_list = list(range(self.vocab_size))
        X = X_seed
        
        h_init = h.copy()
        h = []
        for l in range(4):
            h.append(h_init.copy())

        for _ in range(size - 1):
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            x = X_one_hot.reshape(1, -1)
            y, h[0], _ = self.forward(x, h[0], self.model[0])
            y, h[1], _ = self.forward(y, h[1], self.model[1])
            y, h[2], _ = self.forward(y, h[2], self.model[2])
            y, h[3], _ = self.forward(y, h[3], self.model[3])
            prob = util.softmax(y)
            idx = np.random.choice(idx_list, p=prob.ravel())
            chars.append(self.idx2char[idx])
            X = idx

        return ''.join(chars)

In [53]:
net = RNN(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [54]:
import numpy as np
import impl.utils as util
import impl.constant as c
import copy
from sklearn.utils import shuffle as skshuffle

def get_minibatch(X, y, minibatch_size, shuffle=True):
    minibatches = []

    if shuffle:
        X, y = skshuffle(X, y)

    for i in range(0, X.shape[0], minibatch_size):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]

        minibatches.append((X_mini, y_mini))

    return minibatches

def adam_rnn(nn, X_train, y_train, alpha=0.001, mb_size=256, n_iter=2000, print_after=100):
    minibatches = get_minibatch(X_train, y_train, mb_size, shuffle=False)

    idx = 0
    state = nn.initial_state()
    smooth_loss = -np.log(1.0 / len(set(X_train)))

    M = {k: np.zeros_like(v) for k, v in nn.model[0].items()}
    R = {k: np.zeros_like(v) for k, v in nn.model[0].items()}
    M2 = {k: np.zeros_like(v) for k, v in nn.model[0].items()}
    R2 = {k: np.zeros_like(v) for k, v in nn.model[0].items()}
    M3 = {k: np.zeros_like(v) for k, v in nn.model[0].items()}
    R3 = {k: np.zeros_like(v) for k, v in nn.model[0].items()}    
    M4 = {k: np.zeros_like(v) for k, v in nn.model[0].items()}
    R4 = {k: np.zeros_like(v) for k, v in nn.model[0].items()}
    beta1 = .9
    beta2 = .999

    for iter in range(1, n_iter + 1):
        t = iter

        if idx >= len(minibatches):
            idx = 0
            state = nn.initial_state()

        X_mini, y_mini = minibatches[idx]
        idx += 1

        # Print loss and test sample
        if iter % print_after == 0:
            print('Iter-{} loss: {:.4f}'.format(iter, smooth_loss))
            sample = nn.test(X_mini[0], state)
            print(sample)

        ys, caches = nn.train_step_fwd(X_mini, state)
        loss, grads = nn.train_step_bwd(y_mini, ys, caches)
        smooth_loss = 0.999 * smooth_loss + 0.001 * loss

        for k in grads[0].keys(): #key, value: items
            M[k] = util.exp_running_avg(M[k], grads[0][k], beta1)
            R[k] = util.exp_running_avg(R[k], grads[0][k]**2, beta2)

            m_k_hat = M[k] / (1. - beta1**(t))
            r_k_hat = R[k] / (1. - beta2**(t))

            nn.model[0][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)
    
        for k in grads[0].keys(): #key, value: items
            M2[k] = util.exp_running_avg(M2[k], grads[1][k], beta1)
            R2[k] = util.exp_running_avg(R2[k], grads[1][k]**2, beta2)

            m_k_hat = M2[k] / (1. - beta1**(t))
            r_k_hat = R2[k] / (1. - beta2**(t))

            nn.model[1][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)

        for k in grads[0].keys(): #key, value: items
            M3[k] = util.exp_running_avg(M3[k], grads[2][k], beta1)
            R3[k] = util.exp_running_avg(R3[k], grads[2][k]**2, beta2)

            m_k_hat = M3[k] / (1. - beta1**(t))
            r_k_hat = R3[k] / (1. - beta2**(t))

            nn.model[2][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)

        for k in grads[0].keys(): #key, value: items
            M4[k] = util.exp_running_avg(M4[k], grads[3][k], beta1)
            R4[k] = util.exp_running_avg(R4[k], grads[3][k]**2, beta2)

            m_k_hat = M4[k] / (1. - beta1**(t))
            r_k_hat = R4[k] / (1. - beta2**(t))

            nn.model[3][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)

    return nn

In [55]:
adam_rnn(nn=net, X_train=X, y_train=y, alpha=alpha, mb_size=time_step, n_iter=n_iter, 
                print_after=print_after)

Iter-1000 loss: 3.1179
J-o"c70r%lceen,l ve as el Nrerd lere, deiwoun las the Galal cg ar本uts aifeery in the pnepCuge tee 1s
Iter-2000 loss: 2.1921
 of 6,-"r, Ch6reals Sinothotrthe cintudiudeowes ard Epgh Este War's intt cinnthy instarcentantand Th
Iter-3000 loss: 1.4635
 ferese: dya d thocemmerlyy, The loroon. Acufes in the Inopont rh ind-nouporteke batel Ascan, It cre
Iter-4000 loss: 0.9279
e wolte vatiry. Auggind an erea ac ealira Gaplyupotounan Keseared ind in the Parhentm flomentad coin
Iter-5000 loss: 0.6000
 Sea of Japan, the Game nived ponchafic 1nfrmane Warth ax lipetre Supml mace on Japan has chrdard Ru
Iter-6000 loss: 0.3365
 Japan") is an island 9ourtheveds elano al mons and omgetirims.ichesedheme 4ea ctune sex meuntin is 
Iter-7000 loss: 0.1991
n the world. The Greater Tokyo Area, which includes Tokyo and sever 4ighth largest military budget, 
Iter-8000 loss: 0.0919
. The population of 126 million is the world's tenth largest. Japanese people make up 98.5% of Japan
Iter-900